## 03 Feature Engineering

#### Import relevant libraries

In [1]:
import os
import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt
# %matplotlib inline

#### Load processed dataset

In [2]:
input_dir = os.path.join('..', 'data', 'processed')
data_df = pd.read_csv(os.path.join(input_dir, 'processed_data.csv'))
data_df

,respondent_id,year,bill_gas,bill_other fuel,bill_adjustment,bill_distribution,bill_hydro,bill_nuclear,bill_other,bill_other_fossil,...,investment_value_other_fossil,investment_value_renewables,investment_value_steam,investment_value_transmission,customers,revenues,sales,rps_score,energy_burden,normalized_expenditure
0,2.0,2005,375.031464,7.084629,9.540319,140.598363,24.935923,79.832394,93.020037,115.705892,...,0.0,0.0,323629554.0,116211224.0,1394751.0,4.461784e+09,76622962.0,0.0,0.074330,58.230382
1,2.0,2006,403.967434,8.175744,20.977633,150.312633,25.845165,83.786720,112.218061,136.302804,...,0.0,0.0,313129533.0,110999847.0,1409748.0,4.846311e+09,77363425.0,0.0,0.077073,62.643442
2,2.0,2007,362.674741,9.373432,25.958163,162.860281,26.922500,86.942468,136.816315,146.322099,...,0.0,0.0,221428399.0,141334126.0,1425243.0,5.176492e+09,75564806.0,0.0,0.077817,68.504013
3,2.0,2008,389.722554,12.139279,17.184093,161.259248,25.216835,87.957893,136.354090,157.862545,...,0.0,0.0,661587762.0,134557031.0,1435370.0,5.882591e+09,75436226.0,0.0,0.084360,77.980988
4,2.0,2009,368.234034,10.900991,22.232564,178.909959,25.595063,94.547352,144.596841,136.960261,...,0.0,0.0,479854652.0,159051998.0,1435611.0,5.354824e+09,71369366.0,0.0,0.079537,75.029720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851,428.0,2016,120.751263,294.214436,-116.874290,203.470335,0.000000,0.000000,179.737203,0.000000,...,0.0,0.0,0.0,3297710.0,61923.0,8.727068e+07,977118.0,0.0,0.026651,89.314366
1852,428.0,2017,144.546326,323.702090,-168.226409,206.674057,0.000000,0.000000,207.970206,0.000000,...,0.0,0.0,0.0,3240558.0,62107.0,8.094594e+07,956654.0,0.0,0.022660,84.613601
1853,428.0,2018,337.354284,419.012129,-44.482765,218.418440,0.000000,0.000000,105.759399,0.000000,...,0.0,0.0,0.0,1309415.0,62219.0,9.215581e+07,1009019.0,0.0,0.024630,91.332083
1854,428.0,2019,304.415689,348.490451,-10.125463,228.879953,0.000000,0.000000,68.613095,0.000000,...,0.0,0.0,0.0,9837046.0,62267.0,8.435678e+07,979280.0,0.0,0.022495,86.141633


In [3]:
is_after_2010 = data_df['year'] >= 2010  #inclusive
data_after_2010_df = data_df[is_after_2010]
data_after_2010_df


,respondent_id,year,bill_gas,bill_other fuel,bill_adjustment,bill_distribution,bill_hydro,bill_nuclear,bill_other,bill_other_fossil,...,investment_value_other_fossil,investment_value_renewables,investment_value_steam,investment_value_transmission,customers,revenues,sales,rps_score,energy_burden,normalized_expenditure
5,2.0,2010,375.715469,13.529815,57.306195,207.855781,29.301764,108.447788,146.998675,147.119361,...,0.0,0.0,946788067.0,136373665.0,1436229.0,5.776031e+09,70512676.0,0.0,0.085455,81.914792
6,2.0,2011,310.984887,10.408721,65.922993,198.367910,28.815423,106.466901,123.115743,132.823094,...,0.0,0.0,257254121.0,216313521.0,1434487.0,5.497270e+09,66243667.0,0.0,0.080163,82.985589
7,2.0,2012,251.854838,7.318077,63.958472,193.694724,28.306831,116.294001,135.947289,111.941675,...,0.0,0.0,54826093.0,153362827.0,1440488.0,5.326316e+09,62422449.0,0.0,0.076958,85.326926
8,2.0,2013,305.709743,8.302617,47.394754,196.508342,27.988239,116.667248,137.888028,119.111570,...,0.0,0.0,72715405.0,176758705.0,1444803.0,5.414421e+09,66309626.0,0.0,0.075527,81.653615
9,2.0,2014,321.439930,9.774353,23.646072,206.273280,30.414294,121.274889,148.945759,127.578481,...,0.0,0.0,318747804.0,315617541.0,1450921.0,5.717623e+09,67155314.0,0.0,0.078724,85.140294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851,428.0,2016,120.751263,294.214436,-116.874290,203.470335,0.000000,0.000000,179.737203,0.000000,...,0.0,0.0,0.0,3297710.0,61923.0,8.727068e+07,977118.0,0.0,0.026651,89.314366
1852,428.0,2017,144.546326,323.702090,-168.226409,206.674057,0.000000,0.000000,207.970206,0.000000,...,0.0,0.0,0.0,3240558.0,62107.0,8.094594e+07,956654.0,0.0,0.022660,84.613601
1853,428.0,2018,337.354284,419.012129,-44.482765,218.418440,0.000000,0.000000,105.759399,0.000000,...,0.0,0.0,0.0,1309415.0,62219.0,9.215581e+07,1009019.0,0.0,0.024630,91.332083
1854,428.0,2019,304.415689,348.490451,-10.125463,228.879953,0.000000,0.000000,68.613095,0.000000,...,0.0,0.0,0.0,9837046.0,62267.0,8.435678e+07,979280.0,0.0,0.022495,86.141633


#### Create new features

In [4]:
data_df.columns

Index(['respondent_id', 'year', 'bill_gas', 'bill_other fuel',
       'bill_adjustment', 'bill_distribution', 'bill_hydro', 'bill_nuclear',
       'bill_other', 'bill_other_fossil', 'bill_purchased_power',
       'bill_renewables', 'bill_steam', 'bill_transmission', 'expenditure_gas',
       'expenditure_other fuel', 'expenditure_adjustment',
       'expenditure_distribution', 'expenditure_hydro', 'expenditure_nuclear',
       'expenditure_other', 'expenditure_other_fossil',
       'expenditure_purchased_power', 'expenditure_renewables',
       'expenditure_steam', 'expenditure_transmission', 'housing_units',
       'income', 'asset_value_distribution', 'asset_value_hydro',
       'asset_value_nuclear', 'asset_value_other', 'asset_value_other_fossil',
       'asset_value_renewables', 'asset_value_steam',
       'asset_value_transmission', 'earnings_value_distribution',
       'earnings_value_hydro', 'earnings_value_nuclear',
       'earnings_value_other', 'earnings_value_other_fossil',

In [5]:
# Target variable
normalized_expenditure = data_df[['normalized_expenditure', 'year', 'respondent_id']]
normalized_expenditure

,normalized_expenditure,year,respondent_id
0,58.230382,2005,2.0
1,62.643442,2006,2.0
2,68.504013,2007,2.0
3,77.980988,2008,2.0
4,75.029720,2009,2.0
...,...,...,...
1851,89.314366,2016,428.0
1852,84.613601,2017,428.0
1853,91.332083,2018,428.0
1854,86.141633,2019,428.0


In [6]:
pivot_df = normalized_expenditure.pivot('respondent_id', 'year', 'normalized_expenditure')
pivot_df

year,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
respondent_id,,,,,,,,,,,,,,,,
2.0,58.230382,62.643442,68.504013,77.980988,75.029720,81.914792,82.985589,85.326926,81.653615,85.140294,86.982210,88.978139,92.058930,88.288201,93.006540,93.178476
3.0,83.173105,81.928344,87.053359,135.931113,97.537601,95.479855,106.727718,104.773252,111.581812,104.857121,103.807989,107.545623,114.295111,113.384982,129.031256,115.622007
6.0,41.431378,45.370765,48.360649,54.035775,65.507608,69.367903,66.066211,69.907790,70.177850,83.347911,83.078158,83.816495,83.976018,82.160364,82.109218,83.603337
7.0,68.022316,78.226997,90.008022,96.354499,97.933655,98.997047,99.797568,99.659742,106.576783,104.924547,102.972675,108.199393,114.011420,118.373355,106.538090,108.472808
8.0,59.128989,64.244226,63.496835,73.024992,68.334364,65.970388,67.358778,68.124461,70.739363,65.615480,68.331868,64.718398,68.041586,62.305216,67.897712,73.427857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281.0,68.442735,74.740886,69.839062,71.952150,77.968428,84.955709,80.353551,79.165313,87.303997,88.462387,89.693724,93.783086,91.543274,94.802702,91.590953,92.026476
288.0,97.649578,97.938741,98.184279,105.605201,92.426561,80.022669,75.504112,76.678799,80.392296,90.927839,97.469567,100.126516,88.552988,84.503183,78.797925,88.826783
290.0,106.353331,124.048949,124.031824,130.012682,122.420456,112.604757,103.994461,105.624297,108.344476,121.049820,124.162454,108.254810,116.046904,121.143610,131.633190,130.387188


In [7]:
#t = 10
pivot_df['normalized_expenditure_cagr'] = (pivot_df[2020]/pivot_df[2010])**0.1 - 1
pivot_df

year,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,normalized_expenditure_cagr
respondent_id,,,,,,,,,,,,,,,,,
2.0,58.230382,62.643442,68.504013,77.980988,75.029720,81.914792,82.985589,85.326926,81.653615,85.140294,86.982210,88.978139,92.058930,88.288201,93.006540,93.178476,0.012967
3.0,83.173105,81.928344,87.053359,135.931113,97.537601,95.479855,106.727718,104.773252,111.581812,104.857121,103.807989,107.545623,114.295111,113.384982,129.031256,115.622007,0.019325
6.0,41.431378,45.370765,48.360649,54.035775,65.507608,69.367903,66.066211,69.907790,70.177850,83.347911,83.078158,83.816495,83.976018,82.160364,82.109218,83.603337,0.018841
7.0,68.022316,78.226997,90.008022,96.354499,97.933655,98.997047,99.797568,99.659742,106.576783,104.924547,102.972675,108.199393,114.011420,118.373355,106.538090,108.472808,0.009183
8.0,59.128989,64.244226,63.496835,73.024992,68.334364,65.970388,67.358778,68.124461,70.739363,65.615480,68.331868,64.718398,68.041586,62.305216,67.897712,73.427857,0.010767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281.0,68.442735,74.740886,69.839062,71.952150,77.968428,84.955709,80.353551,79.165313,87.303997,88.462387,89.693724,93.783086,91.543274,94.802702,91.590953,92.026476,0.008027
288.0,97.649578,97.938741,98.184279,105.605201,92.426561,80.022669,75.504112,76.678799,80.392296,90.927839,97.469567,100.126516,88.552988,84.503183,78.797925,88.826783,0.010492
290.0,106.353331,124.048949,124.031824,130.012682,122.420456,112.604757,103.994461,105.624297,108.344476,121.049820,124.162454,108.254810,116.046904,121.143610,131.633190,130.387188,0.014770


In [8]:
target = pivot_df['normalized_expenditure_cagr']
target

respondent_id
2.0      0.012967
3.0      0.019325
6.0      0.018841
7.0      0.009183
8.0      0.010767
           ...   
281.0    0.008027
288.0    0.010492
290.0    0.014770
403.0    0.029475
428.0   -0.024404
Name: normalized_expenditure_cagr, Length: 116, dtype: float64

0. Transmission Investments (as % of investments)
1. Total Investments / sales
2. Renewable Investments (as % of investments)
3. 
4. 

In [9]:
data_after_2010_df.head()

,respondent_id,year,bill_gas,bill_other fuel,bill_adjustment,bill_distribution,bill_hydro,bill_nuclear,bill_other,bill_other_fossil,...,investment_value_other_fossil,investment_value_renewables,investment_value_steam,investment_value_transmission,customers,revenues,sales,rps_score,energy_burden,normalized_expenditure
5,2.0,2010,375.715469,13.529815,57.306195,207.855781,29.301764,108.447788,146.998675,147.119361,...,0.0,0.0,946788067.0,136373665.0,1436229.0,5.776031e+09,70512676.0,0.0,0.085455,81.914792
6,2.0,2011,310.984887,10.408721,65.922993,198.367910,28.815423,106.466901,123.115743,132.823094,...,0.0,0.0,257254121.0,216313521.0,1434487.0,5.497270e+09,66243667.0,0.0,0.080163,82.985589
7,2.0,2012,251.854838,7.318077,63.958472,193.694724,28.306831,116.294001,135.947289,111.941675,...,0.0,0.0,54826093.0,153362827.0,1440488.0,5.326316e+09,62422449.0,0.0,0.076958,85.326926
8,2.0,2013,305.709743,8.302617,47.394754,196.508342,27.988239,116.667248,137.888028,119.111570,...,0.0,0.0,72715405.0,176758705.0,1444803.0,5.414421e+09,66309626.0,0.0,0.075527,81.653615
9,2.0,2014,321.439930,9.774353,23.646072,206.273280,30.414294,121.274889,148.945759,127.578481,...,0.0,0.0,318747804.0,315617541.0,1450921.0,5.717623e+09,67155314.0,0.0,0.078724,85.140294


#### Create DataFrames for Investments

In [10]:
investment_column_names = ['respondent_id', 'investment_value_distribution',
       'investment_value_hydro', 'investment_value_nuclear',
       'investment_value_other', 'investment_value_other_fossil',
       'investment_value_renewables', 'investment_value_steam',
       'investment_value_transmission', 'sales']

In [11]:
# Create a DataFranme of utility investments

sum_investments_sales_df = data_after_2010_df[investment_column_names].groupby(by=["respondent_id"]).sum()
investment_column_names.remove('respondent_id')
sum_investments_sales_df

,investment_value_distribution,investment_value_hydro,investment_value_nuclear,investment_value_other,investment_value_other_fossil,investment_value_renewables,investment_value_steam,investment_value_transmission,sales
respondent_id,,,,,,,,,
2.0,4.122966e+09,401837956.0,9.306870e+08,4.714471e+09,8.419062e+08,5.482545e+07,5.389831e+09,2.662579e+09,711088355.0
3.0,1.593899e+07,11378848.0,0.000000e+00,1.093068e+07,0.000000e+00,0.000000e+00,0.000000e+00,9.275973e+06,4253672.0
6.0,2.242890e+09,100494780.0,0.000000e+00,4.571355e+08,0.000000e+00,0.000000e+00,1.831950e+09,2.279097e+09,428325860.0
7.0,3.157227e+09,0.0,7.497268e+08,2.100995e+09,1.567304e+09,8.982726e+08,1.079611e+09,1.578813e+09,352766341.0
8.0,1.934680e+09,17928575.0,1.159162e+09,8.035705e+08,0.000000e+00,0.000000e+00,1.700975e+08,1.575136e+09,330248085.0
...,...,...,...,...,...,...,...,...,...
281.0,2.146506e+09,0.0,0.000000e+00,5.060051e+08,7.507275e+08,1.802862e+09,1.063686e+09,0.000000e+00,192945960.0
288.0,2.069657e+08,0.0,0.000000e+00,6.311957e+07,2.597997e+06,2.815807e+07,0.000000e+00,1.070905e+08,23726324.0
290.0,1.972978e+08,0.0,0.000000e+00,4.232020e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,13446757.0


In [12]:
sum_investments_sales_df['sum_investments'] = sum_investments_sales_df[investment_column_names].sum(axis=1)
sum_investments_sales_df['sum_investments_generation'] = sum_investments_sales_df['sum_investments'] - sum_investments_sales_df['investment_value_transmission'] - sum_investments_sales_df['investment_value_distribution']
sum_investments_sales_df

,investment_value_distribution,investment_value_hydro,investment_value_nuclear,investment_value_other,investment_value_other_fossil,investment_value_renewables,investment_value_steam,investment_value_transmission,sales,sum_investments,sum_investments_generation
respondent_id,,,,,,,,,,,
2.0,4.122966e+09,401837956.0,9.306870e+08,4.714471e+09,8.419062e+08,5.482545e+07,5.389831e+09,2.662579e+09,711088355.0,1.983019e+10,1.304465e+10
3.0,1.593899e+07,11378848.0,0.000000e+00,1.093068e+07,0.000000e+00,0.000000e+00,0.000000e+00,9.275973e+06,4253672.0,5.177816e+07,2.656320e+07
6.0,2.242890e+09,100494780.0,0.000000e+00,4.571355e+08,0.000000e+00,0.000000e+00,1.831950e+09,2.279097e+09,428325860.0,7.339893e+09,2.817906e+09
7.0,3.157227e+09,0.0,7.497268e+08,2.100995e+09,1.567304e+09,8.982726e+08,1.079611e+09,1.578813e+09,352766341.0,1.148472e+10,6.748676e+09
8.0,1.934680e+09,17928575.0,1.159162e+09,8.035705e+08,0.000000e+00,0.000000e+00,1.700975e+08,1.575136e+09,330248085.0,5.990823e+09,2.481007e+09
...,...,...,...,...,...,...,...,...,...,...,...
281.0,2.146506e+09,0.0,0.000000e+00,5.060051e+08,7.507275e+08,1.802862e+09,1.063686e+09,0.000000e+00,192945960.0,6.462733e+09,4.316227e+09
288.0,2.069657e+08,0.0,0.000000e+00,6.311957e+07,2.597997e+06,2.815807e+07,0.000000e+00,1.070905e+08,23726324.0,4.316582e+08,1.176020e+08
290.0,1.972978e+08,0.0,0.000000e+00,4.232020e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,13446757.0,2.530647e+08,5.576695e+07


#### Create DataFrames for Expenditures

In [13]:
expenditure_column_names = ['respondent_id', 'expenditure_gas',
       'expenditure_other fuel', 'expenditure_adjustment',
       'expenditure_distribution', 'expenditure_hydro', 'expenditure_nuclear',
       'expenditure_other', 'expenditure_other_fossil',
       'expenditure_purchased_power', 'expenditure_renewables',
       'expenditure_steam', 'expenditure_transmission']

In [14]:
sum_expenditures_df = data_after_2010_df[expenditure_column_names].groupby(by=["respondent_id"]).sum()
expenditure_column_names = expenditure_column_names.remove('respondent_id')
sum_expenditures_df

,expenditure_gas,expenditure_other fuel,expenditure_adjustment,expenditure_distribution,expenditure_hydro,expenditure_nuclear,expenditure_other,expenditure_other_fossil,expenditure_purchased_power,expenditure_renewables,expenditure_steam,expenditure_transmission
respondent_id,,,,,,,,,,,,
2.0,5.132308e+09,1.383818e+08,1.154110e+09,3.828276e+09,5.544618e+08,2.282221e+09,2.405275e+09,2.067986e+09,1.489665e+09,1.176959e+07,9.084542e+09,1.950072e+09
3.0,2.882324e+07,2.040340e+08,-6.762004e+07,2.969037e+07,5.227997e+07,0.000000e+00,9.763331e+07,1.528938e+07,4.715983e+07,0.000000e+00,0.000000e+00,1.830169e+07
6.0,2.836928e+09,9.615715e+08,-7.210860e+08,2.290124e+09,1.512317e+08,0.000000e+00,6.603258e+08,2.607276e+08,3.003127e+09,0.000000e+00,6.127269e+09,1.852221e+09
7.0,3.309975e+09,8.289884e+07,-6.541575e+08,3.825312e+09,-1.939861e+06,2.556462e+09,2.064731e+09,3.710783e+09,1.972733e+09,4.508811e+08,3.033181e+09,1.825703e+09
8.0,2.605195e+09,1.507844e+08,-1.236624e+08,1.218794e+09,2.140047e+07,2.131851e+09,1.322763e+09,3.104126e+08,1.545107e+09,0.000000e+00,1.454663e+09,6.925835e+08
...,...,...,...,...,...,...,...,...,...,...,...,...
281.0,3.309155e+09,1.032936e+08,-5.566223e+08,1.163924e+09,0.000000e+00,0.000000e+00,8.970489e+08,7.006013e+08,1.147378e+09,2.256087e+08,1.419898e+09,1.180497e+09
288.0,3.185888e+08,6.995229e+06,-6.080040e+07,1.826391e+08,0.000000e+00,0.000000e+00,1.250259e+08,1.608683e+08,3.889133e+08,8.437912e+06,0.000000e+00,1.411904e+08
290.0,6.297315e+08,6.775422e+08,2.369448e+07,2.231765e+08,0.000000e+00,0.000000e+00,9.934202e+07,4.451915e+04,3.548869e+08,0.000000e+00,0.000000e+00,1.517976e+08


In [15]:
# Experiencing some errors selecting columns, so respondent id is included in this sum.
# It should not be significant due to the magnitude of the numbers we are working with.
sum_expenditures_df['sum_expenditures'] = sum_expenditures_df.sum(axis=1)
sum_expenditures_df

,expenditure_gas,expenditure_other fuel,expenditure_adjustment,expenditure_distribution,expenditure_hydro,expenditure_nuclear,expenditure_other,expenditure_other_fossil,expenditure_purchased_power,expenditure_renewables,expenditure_steam,expenditure_transmission,sum_expenditures
respondent_id,,,,,,,,,,,,,
2.0,5.132308e+09,1.383818e+08,1.154110e+09,3.828276e+09,5.544618e+08,2.282221e+09,2.405275e+09,2.067986e+09,1.489665e+09,1.176959e+07,9.084542e+09,1.950072e+09,3.009907e+10
3.0,2.882324e+07,2.040340e+08,-6.762004e+07,2.969037e+07,5.227997e+07,0.000000e+00,9.763331e+07,1.528938e+07,4.715983e+07,0.000000e+00,0.000000e+00,1.830169e+07,4.255917e+08
6.0,2.836928e+09,9.615715e+08,-7.210860e+08,2.290124e+09,1.512317e+08,0.000000e+00,6.603258e+08,2.607276e+08,3.003127e+09,0.000000e+00,6.127269e+09,1.852221e+09,1.742244e+10
7.0,3.309975e+09,8.289884e+07,-6.541575e+08,3.825312e+09,-1.939861e+06,2.556462e+09,2.064731e+09,3.710783e+09,1.972733e+09,4.508811e+08,3.033181e+09,1.825703e+09,2.217656e+10
8.0,2.605195e+09,1.507844e+08,-1.236624e+08,1.218794e+09,2.140047e+07,2.131851e+09,1.322763e+09,3.104126e+08,1.545107e+09,0.000000e+00,1.454663e+09,6.925835e+08,1.132989e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
281.0,3.309155e+09,1.032936e+08,-5.566223e+08,1.163924e+09,0.000000e+00,0.000000e+00,8.970489e+08,7.006013e+08,1.147378e+09,2.256087e+08,1.419898e+09,1.180497e+09,9.590783e+09
288.0,3.185888e+08,6.995229e+06,-6.080040e+07,1.826391e+08,0.000000e+00,0.000000e+00,1.250259e+08,1.608683e+08,3.889133e+08,8.437912e+06,0.000000e+00,1.411904e+08,1.271859e+09
290.0,6.297315e+08,6.775422e+08,2.369448e+07,2.231765e+08,0.000000e+00,0.000000e+00,9.934202e+07,4.451915e+04,3.548869e+08,0.000000e+00,0.000000e+00,1.517976e+08,2.160216e+09


#### Create DataFrames for Bills

In [16]:
bill_column_names = ['respondent_id', 'bill_gas', 'bill_other fuel',
       'bill_adjustment', 'bill_distribution', 'bill_hydro', 'bill_nuclear',
       'bill_other', 'bill_other_fossil', 'bill_purchased_power',
       'bill_renewables', 'bill_steam', 'bill_transmission']

In [17]:
sum_bills_df = data_after_2010_df[bill_column_names].groupby(by=["respondent_id"]).sum()
bill_column_names = bill_column_names.remove('respondent_id')

# Experiencing some errors selecting columns, so respondent id is included in this sum.
# It should not be significant due to the magnitude of the numbers we are working with.
sum_bills_df['sum_bills'] = sum_bills_df.sum(axis=1)
sum_bills_df

,bill_gas,bill_other fuel,bill_adjustment,bill_distribution,bill_hydro,bill_nuclear,bill_other,bill_other_fossil,bill_purchased_power,bill_renewables,bill_steam,bill_transmission,sum_bills
respondent_id,,,,,,,,,,,,,
2.0,3112.849798,91.867699,722.505017,2397.651783,347.292020,1428.439814,1508.741296,1298.526264,932.665316,7.251702,5702.835851,1221.078770,18771.705330
3.0,1039.013347,9592.164214,-3610.057522,1578.462964,2795.758135,0.000000,5190.673383,804.549527,2512.004816,0.000000,0.000000,974.200009,20876.768872
6.0,2492.989141,934.347519,-694.036696,2205.920362,145.683812,0.000000,635.040939,251.289644,2886.430488,0.000000,5898.707403,1785.539222,16541.911835
7.0,2291.888455,64.010759,-474.081710,2790.104225,-1.473992,1877.509148,1499.625556,2719.986126,1435.407503,324.879991,2216.204948,1328.673630,16072.734640
8.0,3539.049894,204.800539,-160.285125,1616.729997,28.378125,2828.859928,1753.931631,411.205319,2057.549449,0.000000,1931.627131,917.978640,15129.825529
...,...,...,...,...,...,...,...,...,...,...,...,...,...
281.0,5926.665316,187.696145,-1035.572238,2163.386881,0.000000,0.000000,1668.111884,1303.731693,2131.163486,419.041449,2637.799451,2194.358292,17596.382360
288.0,3013.871089,59.328315,-576.979726,1721.097976,0.000000,0.000000,1174.632778,1502.781003,3695.049646,77.144827,0.000000,1320.371195,11987.297102
290.0,6482.254469,6239.562285,259.092912,2436.188675,0.000000,0.000000,1085.240738,0.487741,3887.540777,0.000000,0.000000,1655.217579,22045.585176


## Set Variables

### x1: % of investments that go to transmission and distribution

In [33]:
# x1 is the % of investments that go to transmission
x1_transmission = sum_investments_sales_df['investment_value_transmission'] / sum_investments_sales_df['sum_investments']
x1_transmission

respondent_id
2.0      0.134269
3.0      0.179148
6.0      0.310508
7.0      0.137471
8.0      0.262925
           ...   
281.0    0.000000
288.0    0.248091
290.0    0.000000
403.0    0.204125
428.0    0.248230
Length: 116, dtype: float64

In [32]:
# x1 is the % of investments that go to distribution
x1_distribution = sum_investments_sales_df['investment_value_distribution'] / sum_investments_sales_df['sum_investments']
x1_distribution

respondent_id
2.0      0.207914
3.0      0.307832
6.0      0.305575
7.0      0.274907
8.0      0.322941
           ...   
281.0    0.332136
288.0    0.479467
290.0    0.779634
403.0    0.465455
428.0    0.620644
Length: 116, dtype: float64

### x2: % of expenditures spent on clean energy

In [19]:
# x2 measures the % of expenitures for "clean" energy (renewables, nuclear, and hydro)
expenditure_clean = sum_expenditures_df['expenditure_renewables'] + sum_expenditures_df['expenditure_nuclear'] + sum_expenditures_df['expenditure_hydro']

x2_portfolio = expenditure_clean / sum_expenditures_df['sum_expenditures']
x2_portfolio

respondent_id
2.0      0.094636
3.0      0.122841
6.0      0.008680
7.0      0.135522
8.0      0.190050
           ...   
281.0    0.023523
288.0    0.006634
290.0    0.000000
403.0    0.001236
428.0    0.000000
Length: 116, dtype: float64

### x3a: total investments over sales

In [20]:
# x3a measures the total investments, divided by sales
x3a_total_investment = sum_investments_sales_df['sum_investments']/sum_investments_sales_df['sales']
x3a_total_investment

respondent_id
2.0      27.887101
3.0      12.172580
6.0      17.136237
7.0      32.556154
8.0      18.140371
           ...    
281.0    33.495041
288.0    18.193219
290.0    18.819757
403.0    14.202419
428.0    12.265640
Length: 116, dtype: float64

### x3b: Clean/Fossil Investment Ratio

In [21]:
# x3b measures investments in renewables divided by fossil fuels

#columns: investments without transmission
investments_fossil = sum_investments_sales_df['investment_value_steam'] + sum_investments_sales_df['investment_value_other_fossil']
investments_clean = sum_investments_sales_df['investment_value_renewables'] + sum_investments_sales_df['investment_value_nuclear'] + sum_investments_sales_df['investment_value_hydro']
# x3b_renewables_fossil = sum_investments_sales_df['investment_value_renewables']/investments_fossil
# x3b_renewables_fossil


#renewables divided by total investments

# x2_renewables_total = sum_investments_sales_df['investment_value_renewables']/sum_investments_sales_df['sum_investments']
# x2_renewables_total

x3b_renewable_fossil_investment = (investments_clean - investments_fossil) / sum_investments_sales_df['sum_investments']
x3b_renewable_fossil_investment

respondent_id
2.0     -0.244293
3.0      0.219762
6.0     -0.235897
7.0     -0.086978
8.0      0.168089
           ...   
281.0   -0.001787
288.0    0.059214
290.0    0.000000
403.0    0.028103
428.0    0.000000
Length: 116, dtype: float64

### x4: Whether or not the utility is public or private (TBD)

In [22]:
x4_public_private = 0

### x5: Residential/Total Customers

This is currently done with data from 'housing' and 'customers' columns, but could be done better with the data from customers_sales.csv

In [23]:
customer_names = ['respondent_id', 'housing_units', 'customers']
sum_customers_df = data_after_2010_df[customer_names].groupby(by=["respondent_id"]).sum()
                                      
print(sum_customers_df)

               housing_units   customers
respondent_id                           
2.0               13826176.0  16082769.0
3.0                 157356.0    183423.0
6.0                8929786.0  10540097.0
7.0               11613067.0  13048804.0
8.0                6477428.0   7750180.0
...                      ...         ...
281.0              4471556.0   5606705.0
288.0               918388.0   1035898.0
290.0               718213.0    856027.0
403.0               401883.0    454680.0
428.0               603578.0    682950.0

[116 rows x 2 columns]


In [24]:
x5_residential_ratio = sum_customers_df['housing_units']/sum_customers_df['customers']
x5_residential_ratio

respondent_id
2.0      0.859689
3.0      0.857886
6.0      0.847220
7.0      0.889972
8.0      0.835778
           ...   
281.0    0.797537
288.0    0.886562
290.0    0.839007
403.0    0.883881
428.0    0.883781
Length: 116, dtype: float64

### x6: Nuclear/Clean Bills

In [25]:
# x6 measures the ratio of nuclear to clean bills

bills_clean = sum_bills_df['bill_renewables'] + sum_bills_df['bill_nuclear'] + sum_bills_df['bill_hydro']

x6_nuclear_renewable_ratio = sum_bills_df['bill_nuclear']/bills_clean

for index, value in x6_nuclear_renewable_ratio.items():
    v = x6_nuclear_renewable_ratio[index]
    if np.isnan(v):
        x6_nuclear_renewable_ratio[index] = 0

x6_nuclear_renewable_ratio

respondent_id
2.0      0.801151
3.0      0.000000
6.0      0.000000
7.0      0.853058
8.0      0.990068
           ...   
281.0    0.000000
288.0    0.000000
290.0    0.000000
403.0    0.000000
428.0    0.000000
Length: 116, dtype: float64

### x7: Hydro/Clean Bills

In [26]:
# x7 measures the ratio of hydro to clean bills

x7_hydro_renewable_ratio = sum_bills_df['bill_hydro']/bills_clean

for index, value in x7_hydro_renewable_ratio.items():
    v = x7_hydro_renewable_ratio[index]
    if np.isnan(v):
        x7_hydro_renewable_ratio[index] = 0

x7_hydro_renewable_ratio

respondent_id
2.0      0.194781
3.0      1.000000
6.0      1.000000
7.0     -0.000670
8.0      0.009932
           ...   
281.0    0.000000
288.0    0.000000
290.0    0.000000
403.0    0.000000
428.0    0.000000
Length: 116, dtype: float64

### x8: Regulatory Environment

In [27]:
data_from_2020 = data_after_2010_df[data_after_2010_df['year'] == 2020].copy()
data_year_mean = data_after_2010_df.groupby(['respondent_id']).agg({'rps_score': ['mean']})
data_year_mean = data_year_mean.droplevel(1, axis=1)
x8_regulatory = data_year_mean['rps_score']

In [34]:
model_data = pd.DataFrame(
    {
        'target': target, 
        'x0': x1_transmission,
        'x1': x2_portfolio,
        'x2': x3a_total_investment,
        'x3': x3b_renewable_fossil_investment,
        'x4': x1_distribution,
        'x5': x5_residential_ratio,
        'x6': x6_nuclear_renewable_ratio,
        'x7': x7_hydro_renewable_ratio,
        'x8': x8_regulatory
    }
)
model_data

,target,x0,x1,x2,x3,x4,x5,x6,x7,x8
respondent_id,,,,,,,,,,
2.0,0.012967,0.134269,0.094636,27.887101,-0.244293,0.207914,0.859689,0.801151,0.194781,0.000000
3.0,0.019325,0.179148,0.122841,12.172580,0.219762,0.307832,0.857886,0.000000,1.000000,0.000000
6.0,0.018841,0.310508,0.008680,17.136237,-0.235897,0.305575,0.847220,0.000000,1.000000,10.487586
7.0,0.009183,0.137471,0.135522,32.556154,-0.086978,0.274907,0.889972,0.853058,-0.000670,14.585460
8.0,0.010767,0.262925,0.190050,18.140371,0.168089,0.322941,0.835778,0.990068,0.009932,0.000000
...,...,...,...,...,...,...,...,...,...,...
281.0,0.008027,0.000000,0.023523,33.495041,-0.001787,0.332136,0.797537,0.000000,0.000000,0.736438
288.0,0.010492,0.248091,0.006634,18.193219,0.059214,0.479467,0.886562,0.000000,0.000000,11.250000
290.0,0.014770,0.000000,0.000000,18.819757,0.000000,0.779634,0.839007,0.000000,0.000000,0.000000


#### Check for covariance

In [35]:
corr = model_data.corr()
corr.style.background_gradient(cmap='coolwarm')

,target,x0,x1,x2,x3,x4,x5,x6,x7,x8
target,1.000000,0.055527,0.133518,0.427066,-0.086240,-0.334502,-0.117838,-0.145725,0.073513,-0.019632
x0,0.055527,1.000000,-0.124100,0.330933,0.154705,-0.181224,-0.000576,-0.050566,0.043773,0.234717
x1,0.133518,-0.124100,1.000000,-0.015668,0.338044,-0.429853,-0.117840,0.502643,-0.091421,0.037365
x2,0.427066,0.330933,-0.015668,1.000000,0.013411,-0.144591,0.015622,-0.044540,-0.030186,-0.010414
x3,-0.086240,0.154705,0.338044,0.013411,1.000000,0.245297,0.005392,0.159128,-0.032960,0.139680
x4,-0.334502,-0.181224,-0.429853,-0.144591,0.245297,1.000000,0.283824,0.005925,-0.057839,-0.022730
x5,-0.117838,-0.000576,-0.117840,0.015622,0.005392,0.283824,1.000000,-0.029529,0.077333,0.142404
x6,-0.145725,-0.050566,0.502643,-0.044540,0.159128,0.005925,-0.029529,1.000000,-0.310782,0.099025
x7,0.073513,0.043773,-0.091421,-0.030186,-0.032960,-0.057839,0.077333,-0.310782,1.000000,0.175494
x8,-0.019632,0.234717,0.037365,-0.010414,0.139680,-0.022730,0.142404,0.099025,0.175494,1.000000


#### Filter for training dataset

#### Save model_data file

In [30]:
output_dir = os.path.join('..', 'data', 'processed')

In [31]:
model_data.to_csv(os.path.join(output_dir, 'model_data.csv'), index=False)